In [1]:
import os

# Check current working directory
print(os.getcwd())

# Change to the root directory of your project, if necessary
os.chdir('C:/Users/yashv/Downloads/HADLN_implementation/HADLN_implementation/')
print(os.getcwd())


c:\Users\yashv\Downloads\HADLN_implementation\HADLN_implementation\notebooks
C:\Users\yashv\Downloads\HADLN_implementation\HADLN_implementation


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau, EarlyStopping
from datetime import datetime


In [3]:
def create_bi_gru_model(time_step, num_features, num_classes=4, lstm_units=128, dropout_rate=0.3):
    """
    Function to define a Bi-GRU model for ECG classification.
    
    Args:
        time_step (int): Number of time steps in the input.
        num_features (int): Number of features in the input.
        num_classes (int): Number of output classes.
        lstm_units (int): Number of GRU units for the Bidirectional layers.
        dropout_rate (float): Dropout rate to avoid overfitting.
    
    Returns:
        model (tf.keras.Model): Compiled Bi-GRU model.
    """
    # Input layer
    inputs = tf.keras.layers.Input(shape=(time_step, num_features))
    
    # Bi-GRU layers
    x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=256, return_sequences=True))(inputs)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=128, return_sequences=False))(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    
    # Fully connected layers
    x = tf.keras.layers.Dense(units=128, activation='relu')(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.Dense(units=64, activation='relu')(x)
    
    # Output layer (Softmax for multi-class classification)
    outputs = tf.keras.layers.Dense(units=num_classes, activation='softmax')(x)
    
    # Create model
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    

data pre processing

In [4]:
# Load dataset
csv_file_path = 'data/raw/physionet2017.csv'  # Update with the correct path
data = pd.read_csv(csv_file_path)

# Extract features and labels
X = data.iloc[:, :-2].values  # All columns except the last two are features
y = data.iloc[:, -1].values  # Last column is the label

# One-hot encode labels
y_one_hot = tf.keras.utils.to_categorical(y, num_classes=4)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Reshape input data to match model expectations (batch_size, time_steps, num_features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))


compiling model

In [7]:
# Extract time_step and num_features
time_step = X_train.shape[1]
num_features = X_train.shape[2]

# Build and compile the Bi-GRU model
num_classes = 4
model = create_bi_gru_model(time_step, num_features, num_classes=num_classes)

# Debugging: Print model summary to verify successful creation
if model is not None:
    model.summary()
else:
    raise ValueError("Model creation failed, returned NoneType.")

ValueError: Model creation failed, returned NoneType.

In [5]:
# defining callbacks for training

log_dir = f"logs/tensorboard/BiLSTM/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
checkpoint_callback = ModelCheckpoint(filepath='bilstm_best_model.h5', save_best_only=True, verbose=1)
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-8)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
callbacks = [checkpoint_callback, tensorboard_callback, lr_scheduler, early_stopping]

In [ ]:
# Calculate class weights to handle class imbalance
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weights = dict(enumerate(class_weights))

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    class_weight=class_weights,
    callbacks=callbacks
)